In [1]:
# Let's see what's available
from langchain import agents
print(dir(agents))

['AgentState', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'create_agent', 'factory', 'middleware', 'structured_output']


In [3]:
# Check what langchain packages are actually installed
!pip list | grep langchain

langchain                               1.1.0
langchain-aws                           1.1.0
langchain-classic                       1.0.0
langchain-community                     0.4.1
langchain-core                          1.1.0
langchain-openai                        0.3.5
langchain-text-splitters                1.0.0


In [4]:
# Install only wikipedia
!pip install -q wikipedia


In [5]:
# Import necessary libraries for LangChain 1.1.0
import boto3
from langchain_aws import ChatBedrock
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
import json

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [6]:
# Initialize Amazon Bedrock client
llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name="us-east-1",
    model_kwargs={
        "temperature": 0.7,
        "max_tokens": 2000
    }
)

print("✅ AWS Bedrock LLM initialized successfully!")

✅ AWS Bedrock LLM initialized successfully!


In [7]:
# Test basic LLM call
response = llm.invoke("What is Amazon Bedrock in one sentence?")
print(response.content)

Amazon Bedrock is a fully-managed service that provides a secure and highly available environment for running applications on Amazon Web Services (AWS) using containers.


In [8]:
# Create a simple prompt template and chain using LCEL
prompt_template = PromptTemplate.from_template(
    "Explain {topic} in simple terms for a beginner."
)

chain = prompt_template | llm

# Test the chain
result = chain.invoke({"topic": "LangChain"})
print(result.content)

LangChain is a Python library that helps you build applications that can interact with different types of data sources, such as text, images, and databases, using natural language processing (NLP) and other AI technologies. It's designed to make it easier to build applications that can understand and generate human-like language.

Here's an explanation of LangChain in simple terms:

1. **Data Ingestion**: LangChain allows you to easily load and work with different types of data, such as text files, web pages, PDFs, and even APIs. This data can then be processed and analyzed using various NLP techniques.

2. **Language Models**: LangChain integrates with powerful language models like GPT-3, BERT, and others. These models can be used to generate human-like text, answer questions, summarize information, and more.

3. **Agents and Tools**: LangChain provides a set of pre-built "agents" and "tools" that can perform various tasks like searching the web, querying databases, or performing calc

In [9]:
# Create Wikipedia tool
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

print("✅ Wikipedia tool created!")

✅ Wikipedia tool created!


In [10]:
# Test Wikipedia tool directly
query = "Amazon Web Services"
print(f"🔍 Searching Wikipedia for: {query}\n")
result = wikipedia.run(query)
print(result[:500] + "..." if len(result) > 500 else result)

🔍 Searching Wikipedia for: Amazon Web Services

Page: Amazon Web Services
Summary: Amazon Web Services, Inc. (AWS) is a subsidiary of Amazon that provides on-demand cloud computing platforms and APIs to individuals, companies, and governments, on a metered, pay-as-you-go basis.
Clients often use this in combination with autoscaling (a process that allows a client to use more computing in times of high application usage, and then scale down to reduce costs when there is less traffic). These cloud computing web services provide various services...


In [11]:
# Create a calculator using LLM with structured prompting
calc_prompt = PromptTemplate.from_template(
    """You are a calculator. Solve this math problem and return ONLY the numerical answer.

Problem: {calculation}

Answer:"""
)

calculator_chain = calc_prompt | llm

print("✅ Calculator chain created!")

✅ Calculator chain created!


In [12]:
# Test calculator
calculation = "What is 25% of 847 plus 156?"
print(f"🔍 Calculating: {calculation}\n")
result = calculator_chain.invoke({"calculation": calculation})
print(f"📝 Answer: {result.content}")

🔍 Calculating: What is 25% of 847 plus 156?

📝 Answer: 368


In [20]:
# Create a function that routes to appropriate tools (fixed order)
def ask_agent(question):
    """
    Simple agent that routes questions to Wikipedia or Calculator
    """
    print(f"🤖 Question: {question}\n")
    
    question_lower = question.lower()
    
    # CHECK CALCULATOR FIRST (before Wikipedia)
    if any(word in question_lower for word in ['calculate', 'multiply', 'multiplied', 'divide', 'divided', 'plus', 'minus', 'add', 'subtract', '%', 'percent', 'squared', 'sqrt', 'math', 'times', '*', '+', '-', '/']):
        print("🔧 Tool Selected: Calculator")
        result = calculator_chain.invoke({"calculation": question})
        print(f"✅ Answer: {result.content}\n")
        return result.content
    
    # Route to Wikipedia for factual queries
    elif any(word in question_lower for word in ['who', 'what is', 'where', 'when', 'capital', 'population', 'founded', 'invented', 'created']):
        print("🔧 Tool Selected: Wikipedia")
        try:
            # Extract search term
            search_term = question.replace("?", "").replace("What is", "").replace("Who is", "").replace("Where is", "").strip()
            wiki_info = wikipedia.run(search_term)
            
            # Use LLM to synthesize answer from Wikipedia
            synthesis_prompt = PromptTemplate.from_template(
                """Based on the following information from Wikipedia:

{context}

Answer this question concisely: {question}"""
            )
            chain = synthesis_prompt | llm
            result = chain.invoke({"context": wiki_info[:1500], "question": question})
            print(f"✅ Answer: {result.content}\n")
            return result.content
        except Exception as e:
            print(f"⚠️ Wikipedia error: {e}")
            print("Falling back to LLM...\n")
    
    # Default to direct LLM
    print("🔧 Tool Selected: Direct LLM")
    result = llm.invoke(question)
    print(f"✅ Answer: {result.content}\n")
    return result.content

print("✅ Agent function ready!")

✅ Agent function ready!


In [21]:
ask_agent("What is the capital of France?")

🤖 Question: What is the capital of France?

🔧 Tool Selected: Wikipedia
✅ Answer: The capital of France is Paris.



'The capital of France is Paris.'

In [22]:
ask_agent("What is 144 multiplied by 7?")

🤖 Question: What is 144 multiplied by 7?

🔧 Tool Selected: Calculator
✅ Answer: 1008



'1008'

In [23]:
ask_agent("What is 25% of 847 plus 156?")

🤖 Question: What is 25% of 847 plus 156?

🔧 Tool Selected: Calculator
✅ Answer: 368



'368'

In [24]:
ask_agent("Who invented the Python programming language?")

🤖 Question: Who invented the Python programming language?

🔧 Tool Selected: Wikipedia
✅ Answer: Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language.



'Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language.'

In [25]:
ask_agent("When was Amazon Web Services founded and what does it do?")

🤖 Question: When was Amazon Web Services founded and what does it do?

🔧 Tool Selected: Wikipedia
✅ Answer: Amazon Web Services (AWS) was founded in 2006 as a subsidiary of Amazon.com. It provides a suite of cloud computing services that offer an on-demand computing platform, allowing customers to access storage, computing power, databases and other functionality over the internet on a pay-as-you-go basis.



'Amazon Web Services (AWS) was founded in 2006 as a subsidiary of Amazon.com. It provides a suite of cloud computing services that offer an on-demand computing platform, allowing customers to access storage, computing power, databases and other functionality over the internet on a pay-as-you-go basis.'

In [27]:
# Try your own question!
my_question = "What is the squared root of 256?"
ask_agent(my_question)

🤖 Question: What is the squared root of 256?

🔧 Tool Selected: Calculator
✅ Answer: 16



'16'